In [1]:
import pandas as pd
import re
import numpy as np
import geopandas as gp
import sys
sys.path.append('../../src')
from utils import misc_utils as ct
#%matplotlib inline
import matplotlib.pyplot as plt
import osmnx as ox
import folium
from folium.plugins import MarkerCluster
from folium import IFrame

In [2]:
#############
#Config
############
segmentation_table = 'ten_km'

In [4]:
conn = ct.connect_rds()

# first plot hectopoint segments, with different coloring for each sector
# read in each hectopoint, along with it's assigned cluster
sql = "select s.hectokey, s.hectokey_merged, h.lat, h.long \
from segmentation.{0} as s \
left join rws_clean.unique_hectopunten as h \
on s.hectokey = h.hectokey \
where hectokey_merged  like '%.Li.Li' or hectokey_merged like '%.Re.Re';".format(segmentation_table)
df_hect = pd.read_sql(sql, con=conn)

# extract the road segment value so that we can sort based on that value
df_hect['hecto'] = df_hect['hectokey'].str.extract(r'^\d+\.(\d+)\.\w+\.\w+')[0].astype(int)

# get number of unique segments
sql = "select count(distinct(hectokey_merged)) from segmentation.{0} \
where hectokey_merged  like '%.Li.Li' or hectokey_merged like '%.Re.Re';".format(segmentation_table)
num_seg = pd.read_sql(sql, con=conn).iloc[0]['count']

# generate random colors to use for segmentation
colors = np.random.randint(low=0, high=0xFFFFFF, size=(num_seg,1))

# get ongevallen dataset
sql = "select ongekey, datetime, lat, long from rws_clean.ongevallen;"
df_ong = pd.read_sql(sql, con=conn)
df_ong['datetime'] = df_ong['datetime'].astype(str)


# Get the loop data and add it to the map
sql = "select loop_unique_key, lat, long from rws_clean.flow_hect_link;"
df_loop = pd.read_sql(sql, con=conn)

# Get the weather data and add it to the map
sql = """select "STN" as station_id, "LAT(north)" as lat, "LON(east)" as long from rws_raw.knmi_station \
where "STN" in ('240','260','269','348','356');"""
df_knmi = pd.read_sql(sql, con=conn)

conn.close()

In [4]:
# generates array with the coordinate values for all hectopoints in the segment
def get_seg_coords(df):
    coords = []
    for index, row in df.iterrows():
        coords.append([row['long'], row['lat']])   
    return coords

In [5]:
# Add the background for the map, give two view options, OSM and neutral
# boundary of the image on the map
min_lon = 4.77322
max_lon = 5.57805
min_lat = 51.8252
max_lat = 52.37998994

map_rws = folium.Map(location=[52.2275,5.45671],tiles='openstreetmap')
folium.TileLayer('Stamen Toner').add_to(map_rws)
folium.TileLayer('cartodbpositron').add_to(map_rws)

In [6]:
#Add hectopunten data to map

# NOTE: This code will need to change if segments can consists of more than one road
# generate geojson string for all the coordinates for each segment
df_hect_coords = df_hect.sort_values(['hecto']).groupby(['hectokey_merged']).apply(lambda x: get_seg_coords(x)).reset_index(name='seg_coords')

# write data as geoson structure
fg = folium.FeatureGroup(name='Hectopunten Segments').add_to(map_rws)
for idx, row in df_hect_coords.iterrows():
    geojson_data = {'type':'FeatureCollection', 'features':[]}    
    feature = {'type':'Feature',
'properties':{'name':row['hectokey_merged'], 'strokeColor':'#%06x' % colors[idx][0],
             "popupContent": row['hectokey_merged']},
'geometry':{'type':'LineString',
'coordinates':[]}}
    feature['geometry']['coordinates'] = row['seg_coords']
    geojson_data['features'].append(feature)

# add it to the map
    g = folium.GeoJson(geojson_data,
              name = row['hectokey_merged'],
               style_function=lambda feature: {
                   'color': feature['properties']['strokeColor'],
                   'weight': 2
               })
    g.add_child(folium.Popup(row['hectokey_merged']))
    fg.add_child(g)
#resource: http://geologyandpython.com/geology-of-MI-and-WI.html
#resource: http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb
#resource: http://geoffboeing.com/2015/10/exporting-python-data-geojson/

In [ ]:
# Add ongevallen markers
coords, popups = [], [] 
for i, row in df_ong.iterrows():
    #Append lat and long coordinates to "coords" list
    coords.append([row['lat'], row['long']])
    label = '<br>'.join([row[field] for field in ['ongekey','datetime']])
    popups.append(IFrame(label, width = 300, height = 100))    
#Create a Folium feature group for this layer, since we will be displaying multiple layers
ongevallen = folium.FeatureGroup(name = 'Ongevallen')

ongevallen.add_child(MarkerCluster(locations = coords, popups = popups))
map_rws.add_child(ongevallen)

#reference: http://andrewgaidus.com/leaflet_webmaps_python/
#reference: https://medium.com/@bobhaffner/folium-markerclusters-and-fastmarkerclusters-1e03b01cb7b1 (if need to make faster)

In [ ]:
# Add loop data


lat_loop = [row for row in df_loop['lat']]
lon_loop = [row for row in df_loop['long']]
loop_id =  [row for row in df_loop['loop_unique_key']]

#Create a Folium feature group for this layer, since we will be displaying multiple layers
loops = folium.FeatureGroup(name = 'Flow/Speed Loops')

for i in range(len(loop_id)):
        folium.CircleMarker([lat_loop[i], lon_loop[i]], \
                  radius=1.0, popup=loop_id[i], color = 'red').add_to(loops)

map_rws.add_child(loops)

In [ ]:
# Add weather data

lat_knmi = [row for row in df_knmi['lat']]
lon_knmi = [row for row in df_knmi['long']]
station_id =  [row for row in df_knmi['station_id'].astype(str)]

knmi = folium.FeatureGroup(name = 'Weather Stations')

for i in range(len(station_id)):
        folium.Marker([lat_knmi[i], lon_knmi[i]],
                  icon = folium.Icon(icon = 'cloud', color = 'green'),
                      popup=station_id[i]).add_to(knmi)

map_rws.add_child(knmi)

In [8]:
# add layer control to toggle each data layer as well as background tiles. save as html
folium.LayerControl().add_to(map_rws)
map_rws.save('feature_overlay_map_{0}.html'.format(segmentation_table))